In [1]:
import numpy as np 
import os, json
from os.path import isdir, join 

In [2]:
class score:
    def __init__(self):
        pass
    def get_course_score(self, courseInfo):
        return 0
    def get_schedule_score(self, scheduleInfo):
        return 0 
    def get_compact_score (self, scheduleInfo):
        return 0

In [103]:
mock = [
# 201 Lec
{'course': 'CSCI201', 'ID': '30303', 'type':'Lec','time':'08:00am-09:20am, TTh', 'instructor':'Miller, Jeffrey', 'isFull':False ,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30254', 'type':'Lec','time':'11:00am-12:20pm, TTh', 'instructor':'Grujic, Olivera', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '29998', 'type':'Lec','time':'12:30pm-01:50pm, TTh', 'instructor':'Grujic, Olivera', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '29979', 'type':'Lec','time':'02:00pm-03:20pm, MW' , 'instructor':'Grujic, Olivera', 'isFull':False ,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '29993', 'type':'Lec','time':'10:00am-11:50am, MW' , 'instructor':'Grujic, Olivera', 'isFull':False,'must have':False, 'prefer':False},
# 201 discussion
{'course': 'CSCI201', 'ID': '30239', 'type':'Lab','time':'10:00am-11:50am, W', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30238', 'type':'Lab','time':'12:00pm-01:50pm, W', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30237', 'type':'Lab','time':'02:00pm-03:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30134', 'type':'Lab','time':'04:00pm-05:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30385', 'type':'Lab','time':'04:00pm-05:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30241', 'type':'Lab','time':'06:00pm-07:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '30396', 'type':'Lab','time':'06:00pm-07:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI201', 'ID': '29904', 'type':'Lab','time':'06:00pm-07:50pm, W', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
# 201 quiz
{'course': 'CSCI201', 'ID': '30028', 'type':'Quiz', 'time':'', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
#----------------------------------------------------------
# 356 Lec 
{'course': 'CSCI356', 'ID': '29962', 'type':'Lec','time':'02:00pm-03:20pm, TTh', 'instructor':'Paolieri, Marco', 'isFull':False ,'must have':False, 'prefer':False},
{'course': 'CSCI356', 'ID': '29912', 'type':'Lec','time':'09:30am-10:50am, TTh', 'instructor':'Goodney, Andrew', 'isFull':False ,'must have':False, 'prefer':False},
# 356 Discussino 
{'course': 'CSCI356', 'ID': '30239', 'type':'Dis','time':'02:00pm-03:50pm, F', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI356', 'ID': '30124', 'type':'Dis','time':'10:00am-11:50am, F', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
{'course': 'CSCI356', 'ID': '30239', 'type':'Dis','time':'12:00pm-01:50pm, F', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
# 356 Quiz 
{'course': 'CSCI356', 'ID': '30285', 'type':'Quiz', 'time':'08:00pm-09:50pm, T', 'instructor':'', 'isFull':False,'must have':False, 'prefer':False},
#----------------------------------------------------------
# 467 Lec
{'course': 'CSCI467', 'ID': '30277', 'type':'Lec','time':'02:00pm-03:20pm, MW', 'instructor':'Rajati, Mohammad Reza', 'isFull':False ,'must have':False, 'prefer':False},
# 467 Dis
{'course': 'CSCI467', 'ID': '30278', 'type':'Dis','time':'04:00pm-05:50pm, M', 'instructor':'', 'isFull':False ,'must have':False, 'prefer':False},
{'course': 'CSCI467', 'ID': '30279', 'type':'Dis','time':'04:00pm-05:50pm, T', 'instructor':'', 'isFull':False ,'must have':False, 'prefer':False},
#----------------------------------------------------------
# ENGR100
{'course': 'ENGR100A', 'ID': '28501', 'type':'Lec','time':'01:00pm-01:50pm, F', 'instructor':'', 'isFull':False ,'must have':False, 'prefer':False}

]

In [101]:
from datetime import datetime
from collections import Counter
class greedySearch:
    def get_top_k(self, courseList):
        pass 
    def time_convert(self, course):
        if course['time'] == '':
            return 
        parsed = course['time'].split(',') # get time, day of week
        days = Counter(list(parsed[1]))
        daysNumeric = [] # convert day to numbers 
        if 'M' in days:
            daysNumeric.append(0)
        if 'T' in days and ('h' not in days or days['T']>days['h']):
            daysNumeric.append(1)
        if 'W' in days:
            daysNumeric.append(2)
        if 'T' in days and 'h' in days:
            daysNumeric.append(3)
        if 'F' in days:
            daysNumeric.append (4)
            
        time = parsed[0].split('-') # get start, end time
        startNumeric = [int(time[0].split(':')[0]), int(time[0].split(':')[1][:2])]
        endNumeric = [int(time[1].split(':')[0]), int(time[1].split(':')[1][:2])]
        if 'pm' in time[0] and '12' not in time[0]:
            startNumeric[0] +=12
        if 'pm' in time[1] and '12' not in time[1]:
            endNumeric[0] +=12
        startNumeric = int ((startNumeric[0]-7)*6 + (startNumeric[1]/10))# convert to index
        endNumeric = int ((endNumeric[0]-7)*6 + (endNumeric[1]/10)) # convert to index
        timeNumeric = []
        for i in daysNumeric:
            timeNumeric.append(((i*96) + startNumeric, (i*96) + endNumeric))
        course['timeNumeric'] = timeNumeric

In [104]:
model = greedySearch()
for i in mock:
    model.time_convert(i)
mock

[{'course': 'CSCI201',
  'ID': '30303',
  'type': 'Lec',
  'time': '08:00am-09:20am, TTh',
  'instructor': 'Miller, Jeffrey',
  'isFull': False,
  'must have': False,
  'prefer': False,
  'timeNumeric': [(102, 110), (294, 302)]},
 {'course': 'CSCI201',
  'ID': '30254',
  'type': 'Lec',
  'time': '11:00am-12:20pm, TTh',
  'instructor': 'Grujic, Olivera',
  'isFull': False,
  'must have': False,
  'prefer': False,
  'timeNumeric': [(120, 128), (312, 320)]},
 {'course': 'CSCI201',
  'ID': '29998',
  'type': 'Lec',
  'time': '12:30pm-01:50pm, TTh',
  'instructor': 'Grujic, Olivera',
  'isFull': False,
  'must have': False,
  'prefer': False,
  'timeNumeric': [(129, 137), (321, 329)]},
 {'course': 'CSCI201',
  'ID': '29979',
  'type': 'Lec',
  'time': '02:00pm-03:20pm, MW',
  'instructor': 'Grujic, Olivera',
  'isFull': False,
  'must have': False,
  'prefer': False,
  'timeNumeric': [(42, 50), (234, 242)]},
 {'course': 'CSCI201',
  'ID': '29993',
  'type': 'Lec',
  'time': '10:00am-11:50am